In [1]:
!git clone https://github.com/peppermenta/faceHack

Cloning into 'faceHack'...
remote: Enumerating objects: 6476, done.
remote: Counting objects: 100% (6476/6476), done.
remote: Compressing objects: 100% (6473/6473), done.
remote: Total 6476 (delta 10), reused 6457 (delta 1), pack-reused 0
Receiving objects: 100% (6476/6476), 61.99 MiB | 25.97 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [2]:
!pip3 install -r faceHack/requirements.txt

In [3]:
import torch
from PIL import Image
import numpy as np
import os      
import torchvision         

class FaceDataset(torch.utils.data.Dataset):
  def __init__(self,train=True,type='normal',transform=None):
    super().__init__()
    if(train):
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','train.csv'),delimiter=',',dtype=None,encoding='utf-8')
    else:
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','test.csv'),delimiter=',',dtype=None,encoding='utf-8')

    self.type=type
    self.transform = transform

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, index):
    img_name = self.data[index][0]
    X = Image.open(os.path.join('faceHack','dataset','happy_images','{}.jpg'.format(img_name)))
    label = self.data[index][1]

    if(self.type=='normal'):
      if(label=='NOT smile'):
        y = 0
      else:
        y = 1
    else:
      if(label=='positive smile'):
        y = 1
      else:
        y = 0

    if(self.transform):
      X = self.transform(X)
    return X,y


In [4]:
faceTransform = torchvision.transforms.Compose([
  torchvision.transforms.Resize(224),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.ToTensor(),
])

In [29]:
device = torch.device('cuda')
print(device)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)
model = torchvision.models.resnet18(pretrained=True)

# for param in model.parameters():
#   param.requires_grad = False

in_features = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=in_features,out_features=128),
    torch.nn.Linear(in_features=128,out_features=2)
)

model = model.to(device)

trainDataset = FaceDataset(transform=faceTransform)
testDataset = FaceDataset(train=False,transform=faceTransform)
trainLoader = torch.utils.data.DataLoader(trainDataset,batch_size=32,shuffle=True,num_workers=2)
testLoader = torch.utils.data.DataLoader(testDataset,batch_size=32,shuffle=False,num_workers=2)

cuda


In [30]:
model.train()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
epochs = 20
lambda1 = 0.0001

loss_hist = []
train_acc_hist = []
test_acc_hist = []

for epoch in range(epochs):
  tot_loss = 0
  correct = 0
  for i,data in enumerate(trainLoader):
    X,y = data
    X = X.to(device)
    y = y.to(device)

    out = model(X)

    _,pred = torch.max(out,dim=1)
    correct += (pred==y).sum().item()

    loss = criterion(out,y)
    for p in model.parameters():
      loss += lambda1*torch.sum(p**2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    tot_loss += loss.item()
    if(i%20==0):
      print('Epoch:({},{}): Loss={}'.format(epoch,i,loss.item()))

  loss_hist.append(tot_loss)
  print('Train Accuracy is {}'.format(correct/len(trainDataset)))
  train_acc_hist.append(correct/len(trainDataset))

  test_correct = 0
  with torch.no_grad():
    for X,y in testLoader:
      X = X.to(device)
      y = y.to(device)
      out = model(X)
      _,pred = torch.max(out,dim=1)
      test_correct += (pred==y).sum().item()
  print('Test accuracy is {}'.format(test_correct/len(testDataset)))
  test_acc_hist.append(test_correct/len(testDataset))

import matplotlib.pyplot as plt

plt.plot([i for i in range(epochs)],loss_hist)
plt.show()

plt.plot([i for i in range(epochs)],train_acc_hist)
plt.plot([i for i in range(epochs)],test_acc_hist)
plt.legend(['Train Accuracy','Test Accuracy'])
plt.show()

Epoch:(0,0): Loss=1.3895307779312134
Epoch:(0,20): Loss=1.2433735132217407
Epoch:(0,40): Loss=1.2063820362091064
Epoch:(0,60): Loss=1.3031401634216309
Epoch:(0,80): Loss=1.1980770826339722
Epoch:(0,100): Loss=1.152396559715271
Epoch:(0,120): Loss=1.05098295211792
Epoch:(0,140): Loss=1.0654624700546265
Train Accuracy is 0.7990064168909129
Test accuracy is 0.8373680943513345
Epoch:(1,0): Loss=0.9930232763290405
Epoch:(1,20): Loss=0.8549711108207703
Epoch:(1,40): Loss=1.0228468179702759
Epoch:(1,60): Loss=1.021653652191162
Epoch:(1,80): Loss=1.1394072771072388
Epoch:(1,100): Loss=0.8508842587471008
Epoch:(1,120): Loss=1.0516301393508911
Epoch:(1,140): Loss=1.0992943048477173
Train Accuracy is 0.8664872697164149
Test accuracy is 0.8417132216014898
Epoch:(2,0): Loss=1.1469699144363403
Epoch:(2,20): Loss=0.9340562224388123
Epoch:(2,40): Loss=1.0219371318817139
Epoch:(2,60): Loss=1.0483355522155762
Epoch:(2,80): Loss=1.034873127937317
Epoch:(2,100): Loss=1.0333589315414429
Epoch:(2,120): Loss

KeyboardInterrupt: ignored

In [ ]:
model.eval()